In [1]:
%%capture
# Installs Unsloth, Xformers (Flash Attention) and all other packages!
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps xformers "trl<0.9.0" peft accelerate bitsandbytes

In [2]:
!git clone --recursive https://github.com/ggerganov/llama.cpp

Cloning into 'llama.cpp'...
remote: Enumerating objects: 27723, done.
remote: Counting objects: 100% (171/171), done.
remote: Compressing objects: 100% (122/122), done.
remote: Total 27723 (delta 73), reused 114 (delta 48), pack-reused 27552
Receiving objects: 100% (27723/27723), 51.08 MiB | 19.28 MiB/s, done.
Resolving deltas: 100% (19693/19693), done.
Submodule 'kompute' (https://github.com/nomic-ai/kompute.git) registered for path 'kompute'
Cloning into '/content/llama.cpp/kompute'...
remote: Enumerating objects: 9090, done.        
remote: Counting objects: 100% (225/225), done.        
remote: Compressing objects: 100% (137/137), done.        
remote: Total 9090 (delta 99), reused 172 (delta 78), pack-reused 8865        
Receiving objects: 100% (9090/9090), 17.58 MiB | 29.80 MiB/s, done.
Resolving deltas: 100% (5706/5706), done.
Submodule path 'kompute': checked out '4565194ed7c32d1d2efa32ceab4d3c6cae006306'


In [3]:
!cd llama.cpp && make clean && make all -j

I ccache not found. Consider installing it for faster compilation.
I llama.cpp build info: 
I UNAME_S:   Linux
I UNAME_P:   x86_64
I UNAME_M:   x86_64
I CFLAGS:    -I. -Icommon -D_XOPEN_SOURCE=600 -D_GNU_SOURCE -DNDEBUG -DGGML_USE_OPENMP -DGGML_USE_LLAMAFILE  -std=c11   -fPIC -O3 -Wall -Wextra -Wpedantic -Wcast-qual -Wno-unused-function -Wshadow -Wstrict-prototypes -Wpointer-arith -Wmissing-prototypes -Werror=implicit-int -Werror=implicit-function-declaration -pthread -march=native -mtune=native -fopenmp -Wdouble-promotion 
I CXXFLAGS:  -std=c++11 -fPIC -O3 -Wall -Wextra -Wpedantic -Wcast-qual -Wno-unused-function -Wmissing-declarations -Wmissing-noreturn -pthread -fopenmp  -march=native -mtune=native -Wno-array-bounds -Wno-format-truncation -Wextra-semi -I. -Icommon -D_XOPEN_SOURCE=600 -D_GNU_SOURCE -DNDEBUG -DGGML_USE_OPENMP -DGGML_USE_LLAMAFILE 
I NVCCFLAGS: -std=c++11 -O3 
I LDFLAGS:    
I CC:        cc (Ubuntu 11.4.0-1ubuntu1~22.04) 11.4.0
I CXX:       c++ (Ubuntu 11.4.0-1ubuntu1~

In [4]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


config.json:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

==((====))==  Unsloth: Fast Llama patching release 2024.6
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.0+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. Xformers = 0.0.26.post1. FA = False.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/172 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/464 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [5]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

from datasets import load_dataset
dataset = load_dataset("Antonio27/kidstopics", split = "train")
dataset = dataset.map(formatting_prompts_func, batched = True,)

Generating train split:   0%|          | 0/10260 [00:00<?, ? examples/s]

Map:   0%|          | 0/10260 [00:00<?, ? examples/s]

In [6]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

Unsloth 2024.6 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.
/usr/local/lib/python3.10/dist-packages/multiprocess/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Map (num_proc=2):   0%|          | 0/10260 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


In [7]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 10,260 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 60
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
1,2.135400
2,1.989400
3,1.942700
4,2.052700
5,1.883000
6,1.797000
7,1.711500
8,1.669800
9,1.639600
10,1.586000


In [8]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

question = "What is a rainbow?"

FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        f"Your main task is to answer children's questions or tell stories to children. When you use any complex or difficult words, please explain them using very simple words that a 3-year-old child can understand. After explaining the concept, you can introduce the complex word. Also, ensure that your answers and stories are suitable and engaging for a 3-year-old.\n\nQuestion: {question}", # instruction
        "", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 2048)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Your main task is to answer children's questions or tell stories to children. When you use any complex or difficult words, please explain them using very simple words that a 3-year-old child can understand. After explaining the concept, you can introduce the complex word. Also, ensure that your answers and stories are suitable and engaging for a 3-year-old.

Question: What is a rainbow?

### Input:


### Response:
A rainbow is a beautiful sight in the sky that looks like a colorful arch or band. It is formed when sunlight shines through raindrops in the air. The different colors in a rainbow are caused by the way light is reflected and bent as it passes through the raindrops. The colors are usually arranged in a specific order: red, orange, yellow, green, blue, indigo, and violet. These colors ar

In [9]:
model.save_pretrained("llama3_for_sugar")
tokenizer.save_pretrained("llama3_for_sugar")

('llama3_for_sugar/tokenizer_config.json',
 'llama3_for_sugar/special_tokens_map.json',
 'llama3_for_sugar/tokenizer.json')

In [10]:
hf_token="hf_token"
from huggingface_hub import login
login(token=hf_token, add_to_git_credential=True)

Token is valid (permission: fineGrained).
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [11]:
model.push_to_hub("Antonio27/llama3_for_sugar", token = hf_token)
tokenizer.push_to_hub("Antonio27/llama3_for_sugar", token = hf_token)

README.md:   0%|          | 0.00/576 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/168M [00:00<?, ?B/s]

Saved model to https://huggingface.co/Antonio27/llama3_for_sugar


In [15]:
model.push_to_hub_merged("Antonio27/llama3_for_sugar", tokenizer, save_method = "merged_4bit_forced", token = hf_token)

Unsloth: Merging 4bit and LoRA weights to 4bit...
This might take 5 minutes...


/usr/local/lib/python3.10/dist-packages/peft/tuners/lora/bnb.py:325: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


Done.
Unsloth: Saving 4bit Bitsandbytes model. Please wait...


Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.05G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.65G [00:00<?, ?B/s]

README.md:   0%|          | 0.00/582 [00:00<?, ?B/s]

Saved merged_4bit model to https://huggingface.co/Antonio27/llama3_for_sugar


In [17]:
model.push_to_hub("Antonio27/llama3_adapter_for_sugar", token = hf_token)
tokenizer.push_to_hub("Antonio27/llama3_adapter_for_sugar", token = hf_token)

README.md:   0%|          | 0.00/576 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

Saved model to https://huggingface.co/Antonio27/llama3_adapter_for_sugar


In [19]:
model.push_to_hub_merged("Antonio27/llama3-8b-4-bit-for-sugar", tokenizer, save_method = "merged_4bit_forced", token = hf_token)
tokenizer.push_to_hub("Antonio27/llama3-8b-4-bit-for-sugar", token = hf_token)

Unsloth: Merging 4bit and LoRA weights to 4bit...
This might take 5 minutes...
Done.
Unsloth: Saving 4bit Bitsandbytes model. Please wait...


Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.05G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.65G [00:00<?, ?B/s]

README.md:   0%|          | 0.00/582 [00:00<?, ?B/s]

Saved merged_4bit model to https://huggingface.co/Antonio27/llama3-8b-4-bit-for-sugar


In [20]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

llama_model, llama_tokenizer = FastLanguageModel.from_pretrained(
    model_name = "Antonio27/llama3-8b-4-bit-for-sugar",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

config.json:   0%|          | 0.00/1.14k [00:00<?, ?B/s]

==((====))==  Unsloth: Fast Llama patching release 2024.6
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.0+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. Xformers = 0.0.26.post1. FA = False.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


model.safetensors.index.json:   0%|          | 0.00/132k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.65G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.05G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/172 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/464 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [21]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = llama_tokenizer.eos_token

FastLanguageModel.for_inference(llama_model)

inputs = llama_tokenizer(
[
    alpaca_prompt.format(
        f"Your main task is to answer children's questions or tell stories to children. When you use any complex or difficult words, please explain them using very simple words that a 3-year-old child can understand. After explaining the concept, you can introduce the complex word. Also, ensure that your answers and stories are suitable and engaging for a 3-year-old.\n\nQuestion: {question}", # instruction
        "", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = llama_model.generate(**inputs, max_new_tokens = 256, use_cache = True)
decoded_outputs = llama_tokenizer.batch_decode(outputs)

response_text = decoded_outputs[0]
response_start = response_text.find("### Response:") + len("### Response:")
initial_response = response_text[response_start:].strip()
end_of_text_token = "<|end_of_text|>"
if end_of_text_token in initial_response:
    initial_response = initial_response.split(end_of_text_token)[0].strip()

print("Initial Response:", initial_response)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Initial Response: A rainbow is a beautiful and magical sight in the sky. It is caused by the sun's rays reflecting off tiny water droplets in the air. The rainbow is made up of seven colors, which are red, orange, yellow, green, blue, indigo, and violet. The colors appear in a specific order, with red being the first color and violet being the last. The rainbow is a symbol of hope and good luck in many cultures. It is also a reminder that no matter how dark and dreary the sky may seem, there is always beauty to be found. So, if you ever see a rainbow, take a moment to appreciate its beauty and remember that there is always hope and good luck to be found, no matter how dark and dreary things may seem.
